In [2]:
import pandas as pd
from pandas import DataFrame

### READ GDP DATA

In [3]:
# load rawData
rgdpPath = r'..\datasets\Regional gross domestic product(all ITL).xlsx'

def loadRawData(sheetName="Table 1", index='Region name'):
    rawData = pd.read_excel(rgdpPath, sheet_name=f"{sheetName}", skiprows=1, header=0, index_col=index)
    rawData.index = [item if item !="Yorkshire and The Humber" else "Yorkshire and the Humber" for item in rawData.index]
    rawData.columns = [int(item) if item.startswith('1') or item.startswith('2') else item for item in rawData.columns]
    return rawData
    # print(rawData.index)
    # print(rawData.columns)

loadRawData('Table 5')

,ITL,ITL code,1998,1999,2000,2001,2002,2003,2004,2005,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
United Kingdom,UK,UK,998501,1043248,1100752,1146135,1191439,1258334,1322637,1398749,...,1781361,1862514,1916451,1991645,2082482,2152304,2233921,2104288,2284079,2506170
England,Other,TLB,849042,888297,932187,970734,1012169,1068436,1121038,1183598,...,1515353,1591059,1647017,1714442,1789758,1846878,1918193,1812695,1969953,2161593
North East,ITL1,TLC,32636,33336,34400,35897,38189,40307,43493,45450,...,54425,56044,57987,59034,60817,62132,64599,60217,65438,71763
Tees Valley and Durham,ITL2,TLC1,13956,14573,15081,15427,16198,17301,18473,19130,...,22920,23556,24432,24661,24972,25856,26636,25776,27696,30082
Hartlepool and Stockton-on-Tees,ITL3,TLC11,3739,3920,4033,4092,4312,4603,4919,5088,...,6376,6532,6914,7125,7080,7162,7476,7357,7807,8486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Lisburn and Castlereagh,ITL3,TLN0E,1648,1744,1874,1986,2039,2182,2249,2392,...,2952,3065,3228,3413,3503,3697,3934,3693,4060,4443
Mid and East Antrim,ITL3,TLN0F,2126,2228,2431,2361,2431,2482,2636,2640,...,3319,3274,3599,3539,3220,2808,2943,2797,3021,3253
Fermanagh and Omagh,ITL3,TLN0G,1243,1311,1392,1462,1515,1631,1713,1857,...,2045,2181,2205,2355,2472,2585,2703,2604,2878,3090
Extra-Regio,Other,TLZ,11900,13370,20063,19899,17560,17997,19264,22541,...,21783,17659,10504,10510,14758,20187,18510,12346,12105,15620


In [4]:
def getDataByITL(data:DataFrame, ITL="ITL3", startYear=1998, endYear=2022, isOriginalFormat=True):
    """
    query data by ITL
    ----------
    :param data: raw data
    :param ITL: ITL code
    :param isOriginalFormat: return data format
        True: original data format
        False: transformed data format
    """
    # filter ITL level
    subData = data.copy(deep=True)
    if ITL == "all":
        pass
    else:
        subData = data.query(f"ITL == '{ITL}'").copy()

    # filter year
    if startYear and endYear:
        subData.drop(columns=[year for year in range(1998, startYear)], axis=1, inplace=True)
        subData.drop(columns=[year for year in range(endYear+1, 2023)], axis=1, inplace=True)

    # transform data format
    if not isOriginalFormat:
        subData.reset_index(inplace=True)
        subData.rename(columns={'index': 'Region Name'}, inplace=True)

        subData = subData.melt(
            id_vars=['Region Name', 'ITL', 'ITL code'],
            value_vars=[y for y in range(startYear, endYear+1)],
            var_name='year',
            value_name='value'
        )
    print("getDataByITL shape:", subData.shape)
    return subData

datasheet = loadRawData(sheetName='Table 5')
ITL3 = getDataByITL(data=datasheet, ITL='ITL3', isOriginalFormat=False)
ITL3.head()

getDataByITL shape: (4475, 5)


,Region Name,ITL,ITL code,year,value
0,Hartlepool and Stockton-on-Tees,ITL3,TLC11,1998,3739
1,South Teesside,ITL3,TLC12,1998,3159
2,Darlington,ITL3,TLC13,1998,1675
3,Durham CC,ITL3,TLC14,1998,5382
4,Northumberland,ITL3,TLC21,1998,3473


### READ LAD DATA

In [5]:
ladPath = r'..\datasets\Local_Authority_District_(April_2023)_to_LAU1_to_ITL3_to_ITL2_to_ITL1_(January_2021)Lookup.csv'
lad = pd.read_csv(ladPath)
lad

,LAD23CD,LAD23NM,LAU121CD,LAU121NM,ITL321CD,ITL321NM,ITL221CD,ITL221NM,ITL121CD,ITL121NM,ObjectId
0,E06000001,Hartlepool,E06000001,Hartlepool,TLC11,Hartlepool and Stockton-on-Tees,TLC1,Tees Valley and Durham,TLC,North East (England),1
1,E06000004,Stockton-on-Tees,E06000004,Stockton-on-Tees,TLC11,Hartlepool and Stockton-on-Tees,TLC1,Tees Valley and Durham,TLC,North East (England),2
2,E06000002,Middlesbrough,E06000002,Middlesbrough,TLC12,South Teesside,TLC1,Tees Valley and Durham,TLC,North East (England),3
3,E06000003,Redcar and Cleveland,E06000003,Redcar and Cleveland,TLC12,South Teesside,TLC1,Tees Valley and Durham,TLC,North East (England),4
4,E06000005,Darlington,E06000005,Darlington,TLC13,Darlington,TLC1,Tees Valley and Durham,TLC,North East (England),5
...,...,...,...,...,...,...,...,...,...,...,...
383,E07000138,Lincoln,E07000138,Lincoln,TLF30,Lincolnshire CC,TLF3,Lincolnshire,TLF,East Midlands (England),384
384,E07000139,North Kesteven,E07000139,North Kesteven,TLF30,Lincolnshire CC,TLF3,Lincolnshire,TLF,East Midlands (England),385
385,E07000140,South Holland,E07000140,South Holland,TLF30,Lincolnshire CC,TLF3,Lincolnshire,TLF,East Midlands (England),386
386,E07000141,South Kesteven,E07000141,South Kesteven,TLF30,Lincolnshire CC,TLF3,Lincolnshire,TLF,East Midlands (England),387


In [6]:
lad = pd.read_csv(ladPath, usecols=['ITL321CD', 'ITL321NM', 'LAD23CD', 'LAD23NM'])
print(lad.shape)
lad.head()

(388, 4)


,LAD23CD,LAD23NM,ITL321CD,ITL321NM
0,E06000001,Hartlepool,TLC11,Hartlepool and Stockton-on-Tees
1,E06000004,Stockton-on-Tees,TLC11,Hartlepool and Stockton-on-Tees
2,E06000002,Middlesbrough,TLC12,South Teesside
3,E06000003,Redcar and Cleveland,TLC12,South Teesside
4,E06000005,Darlington,TLC13,Darlington


In [7]:
# get how many ILT325NM in LAD data
print(lad['ITL321CD'].nunique())
print(lad['ITL321NM'].nunique())
print(lad['LAD23CD'].nunique())
print(lad['LAD23NM'].nunique())


179
179
361
361


### match LAD with GDP

In [8]:
# economic data
ITL3 = getDataByITL(data=loadRawData('Table 5'), ITL='ITL3', startYear=1998, endYear=2022, isOriginalFormat=False)
print(ITL3.shape)
ITL3

getDataByITL shape: (4475, 5)
(4475, 5)


,Region Name,ITL,ITL code,year,value
0,Hartlepool and Stockton-on-Tees,ITL3,TLC11,1998,3739
1,South Teesside,ITL3,TLC12,1998,3159
2,Darlington,ITL3,TLC13,1998,1675
3,Durham CC,ITL3,TLC14,1998,5382
4,Northumberland,ITL3,TLC21,1998,3473
...,...,...,...,...,...
4470,Causeway Coast and Glens,ITL3,TLN0C,2022,3019
4471,Antrim and Newtownabbey,ITL3,TLN0D,2022,4310
4472,Lisburn and Castlereagh,ITL3,TLN0E,2022,4443
4473,Mid and East Antrim,ITL3,TLN0F,2022,3253


In [9]:
# get how many unique ITL3 code
print(ITL3['ITL code'].nunique())


179


In [10]:
# merge economic data with LDA
ITL3_with_LDA = pd.merge(ITL3, lad, left_on='ITL code', right_on='ITL321CD', how='left')
print(ITL3_with_LDA.shape)
ITL3_with_LDA

(9700, 9)


,Region Name,ITL,ITL code,year,value,LAD23CD,LAD23NM,ITL321CD,ITL321NM
0,Hartlepool and Stockton-on-Tees,ITL3,TLC11,1998,3739,E06000001,Hartlepool,TLC11,Hartlepool and Stockton-on-Tees
1,Hartlepool and Stockton-on-Tees,ITL3,TLC11,1998,3739,E06000004,Stockton-on-Tees,TLC11,Hartlepool and Stockton-on-Tees
2,South Teesside,ITL3,TLC12,1998,3159,E06000002,Middlesbrough,TLC12,South Teesside
3,South Teesside,ITL3,TLC12,1998,3159,E06000003,Redcar and Cleveland,TLC12,South Teesside
4,Darlington,ITL3,TLC13,1998,1675,E06000005,Darlington,TLC13,Darlington
...,...,...,...,...,...,...,...,...,...
9695,Causeway Coast and Glens,ITL3,TLN0C,2022,3019,N09000004,Causeway Coast and Glens,TLN0C,Causeway Coast and Glens
9696,Antrim and Newtownabbey,ITL3,TLN0D,2022,4310,N09000001,Antrim and Newtownabbey,TLN0D,Antrim and Newtownabbey
9697,Lisburn and Castlereagh,ITL3,TLN0E,2022,4443,N09000007,Lisburn and Castlereagh,TLN0E,Lisburn and Castlereagh
9698,Mid and East Antrim,ITL3,TLN0F,2022,3253,N09000008,Mid and East Antrim,TLN0F,Mid and East Antrim


In [11]:
# find unmatched ITL code
unmatched = ITL3_with_LDA[ITL3_with_LDA['LAD23CD'].isnull()].copy()
print(unmatched.shape)

(0, 9)


In [12]:
# merged data is larger than berfore
# select the first in the table
ITL3_with_LDA = ITL3_with_LDA.groupby(['ITL code', 'year']).agg({
    'Region Name': 'first',
    'ITL': 'first',
    'value': 'first',
    'LAD23CD': 'first',
    'ITL321NM': 'first',
}).reset_index()
print(ITL3_with_LDA.shape)
ITL3_with_LDA

(4475, 7)


,ITL code,year,Region Name,ITL,value,LAD23CD,ITL321NM
0,TLC11,1998,Hartlepool and Stockton-on-Tees,ITL3,3739,E06000001,Hartlepool and Stockton-on-Tees
1,TLC11,1999,Hartlepool and Stockton-on-Tees,ITL3,3920,E06000001,Hartlepool and Stockton-on-Tees
2,TLC11,2000,Hartlepool and Stockton-on-Tees,ITL3,4033,E06000001,Hartlepool and Stockton-on-Tees
3,TLC11,2001,Hartlepool and Stockton-on-Tees,ITL3,4092,E06000001,Hartlepool and Stockton-on-Tees
4,TLC11,2002,Hartlepool and Stockton-on-Tees,ITL3,4312,E06000001,Hartlepool and Stockton-on-Tees
...,...,...,...,...,...,...,...
4470,TLN0G,2018,Fermanagh and Omagh,ITL3,2585,N09000006,Fermanagh and Omagh
4471,TLN0G,2019,Fermanagh and Omagh,ITL3,2703,N09000006,Fermanagh and Omagh
4472,TLN0G,2020,Fermanagh and Omagh,ITL3,2604,N09000006,Fermanagh and Omagh
4473,TLN0G,2021,Fermanagh and Omagh,ITL3,2878,N09000006,Fermanagh and Omagh


In [13]:
ITL3_with_LDA.isna().any()

ITL code       False
year           False
Region Name    False
ITL            False
value          False
LAD23CD        False
ITL321NM       False
dtype: bool

In [14]:
# check if Region Name == ITL321NM
ITL3_with_LDA[ITL3_with_LDA['Region Name'] != ITL3_with_LDA['ITL321NM']][['Region Name', 'ITL321NM']].drop_duplicates()

,Region Name,ITL321NM
75,Durham CC,Durham
1325,Shropshire CC,Shropshire
2550,Buckinghamshire CC,Buckinghamshire
3675,"Inverness and Nairn, Moray, Badenoch and Strat...","Inverness and Nairn, Moray, and Badenoch and S..."


In [15]:
# unique region name, ITL code
print(ITL3_with_LDA['Region Name'].nunique())
print(ITL3_with_LDA['ITL code'].nunique())

179
179


In [16]:
# drop redundant columns
ITL3_with_LDA.drop(columns=['ITL321NM'], inplace=True)
print(ITL3_with_LDA.shape)
ITL3_with_LDA.head()

(4475, 6)


,ITL code,year,Region Name,ITL,value,LAD23CD
0,TLC11,1998,Hartlepool and Stockton-on-Tees,ITL3,3739,E06000001
1,TLC11,1999,Hartlepool and Stockton-on-Tees,ITL3,3920,E06000001
2,TLC11,2000,Hartlepool and Stockton-on-Tees,ITL3,4033,E06000001
3,TLC11,2001,Hartlepool and Stockton-on-Tees,ITL3,4092,E06000001
4,TLC11,2002,Hartlepool and Stockton-on-Tees,ITL3,4312,E06000001
